# Plan de trabajo

Dado que en nuestro equipo no tenemos demasiado tiempo para analizar distintas fuentes vamos a priorizar un Producto Mínimo Viable (MVP) que podamos hacer crecer con nueva disposición de datos.

## Ingeniería de datos

### Análisis de las fuentes

Apoyándonos en la tarea realizada previamente sobre bizkaia hemos analizado la fuente y corregido algunos problemas de formato. La idea es poder extender este modelo a posteriori.

### Esquema de datos

Tenemos **PARTIDOS** que han sido **VOTADOS** en distintos **AMBITOS**. El grano más fino que trabajaremos será el de una **CITA ELECTORAL** dado un **AÑO**, **PARTIDO** y **AMBITO**. Sabemos que puede haber más citas electorales al año y algunas veces los datos nos vendrán por código postal. En nuestro caso hemos decidido tenerlas a ese nivel mínimo de agrupación.

Los partidos estarán categorizados por **ORIENTACIÓN** (Izquierda y Derecha) y los ámbitos por **PROVINCIA**. Tenemos información adicional sobre cada convocatoria en cada provincia relativo a votos **ESCRUTADOS**, **EN BLANCO**, etc...

Con esto podemos tener un esquema de tablas y relaciones claras entre ellas.

![modelo](./img/modelo-datos.png)

# Conexión

Creamos una base de datos MySQL en [Aiven.io](https://aiven.io/) y ahora nos conectamos a ella. 

In [1]:
# !pip install pymysql

Para no guardar las contraseñas en nuestro notebook usaremos un fichero local llamado _.env_ con este contenido.

```
SERVIDOR=xxxx
USUARIO=yyyy
PASSWORD=zzzz
PUERTO=cccc
BASE_DE_DATOS=bbbbb
```

De este modo podremos invocar estos datos como meras variables de entorno.

In [77]:
# !pip install python-dotenv

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [6]:
import os
from sqlalchemy import create_engine

usuario=os.environ.get("USUARIO")
password=os.environ.get("PASSWORD")
puerto=os.environ.get("PUERTO")
servidor=os.environ.get("SERVIDOR")
db=os.environ.get("BASE_DE_DATOS")

engine = create_engine(f"mysql+pymysql://{usuario}:{password}@{servidor}:{puerto}/{db}")
con = engine.connect()

Podemos comprobar que está vacía.

In [7]:
import pandas as pd

pd.read_sql_query("SELECT * FROM information_schema.tables WHERE TABLE_SCHEMA = 'elecciones'", con=con)

,TABLE_CATALOG,TABLE_SCHEMA,TABLE_NAME,TABLE_TYPE,ENGINE,VERSION,ROW_FORMAT,TABLE_ROWS,AVG_ROW_LENGTH,DATA_LENGTH,...,INDEX_LENGTH,DATA_FREE,AUTO_INCREMENT,CREATE_TIME,UPDATE_TIME,CHECK_TIME,TABLE_COLLATION,CHECKSUM,CREATE_OPTIONS,TABLE_COMMENT


# Datos

Necesitamos obtener los datos de los votos en Bizkaia.

In [4]:
import requests
import pandas as pd

response = requests.get("https://www.opendatabizkaia.eus/es/dump/e9c5f672-ec3f-4094-a230-9dd022b28705/elecciones-europeas-2019?format=json")
datos_json = response.json()
datos_df = pd.DataFrame(datos_json.get("records"))
datos_df

,ZENTSUA/CENSO,M.C.R.,EREMUAK/AMBITOS,CPE,ADN,IGRE,PCPE-PCPC-P,ALTER,PCTE-ELAK,I.FEM,...,PACT,AZALPENA/CONCEPTO,PP,SAIN,PODEMOS-IU,PUM+J,R0-LV-GVE,CXE,_id,VOX
0,911770,110,BIZKAIA,507,121,153,619,339,384,1238,...,242,BOTOAK/VOTOS,36178,99,67161,549,994,575,1,7118
1,911770,"0,02",BIZKAIA,"0,09","0,02","0,03","0,1","0,06","0,06","0,21",...,"0,04",BOTOEN %/% DE VOTOS,"6,11","0,02","11,35","0,09","0,17","0,1",2,"1,2"
2,5900,17,ABADIÑO,2,1,0,1,0,1,5,...,2,BOTOAK/VOTOS,155,0,403,2,7,0,3,35
3,5900,"0,42",ABADIÑO,"0,05","0,02",0,"0,02",0,"0,02","0,12",...,"0,05",BOTOEN %/% DE VOTOS,"3,86",0,"10,03","0,05","0,17",0,4,"0,87"
4,7769,0,ABANTO Y CIERVANA-ABANTO ZIERBENA,1,2,1,3,1,3,3,...,6,BOTOAK/VOTOS,94,1,844,4,7,3,5,46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,860,0,ZEBERIO,0,0,0,0,0,0,"0,14",...,0,BOTOEN %/% DE VOTOS,"0,43",0,"4,99",0,0,0,222,"0,29"
222,1226,0,ZIERBENA,0,0,0,0,0,0,1,...,0,BOTOAK/VOTOS,13,0,69,0,1,0,223,3
223,1226,0,ZIERBENA,0,0,0,0,0,0,"0,11",...,0,BOTOEN %/% DE VOTOS,"1,46",0,"7,75",0,"0,11",0,224,"0,34"
224,346,0,ZIORTZA-BOLIBAR,0,0,0,0,0,0,0,...,0,BOTOAK/VOTOS,2,0,8,0,0,0,225,0


In [5]:
# Columnas a eliminar
eliminar=["AZALPENA/CONCEPTO","_id"]

# Totales
datos_cantidad_df = datos_df[datos_df["AZALPENA/CONCEPTO"] == "BOTOAK/VOTOS"].copy()
datos_cantidad_df.drop(columns=eliminar, inplace=True)

# %
datos_perc_df = datos_df[~(datos_df["AZALPENA/CONCEPTO"] == "BOTOAK/VOTOS")].copy()
datos_perc_df.drop(columns=eliminar, inplace=True)

In [6]:
datos_cantidad_df

,ZENTSUA/CENSO,M.C.R.,EREMUAK/AMBITOS,CPE,ADN,IGRE,PCPE-PCPC-P,ALTER,PCTE-ELAK,I.FEM,...,CONTIGO,JUNTS,PACT,PP,SAIN,PODEMOS-IU,PUM+J,R0-LV-GVE,CXE,VOX
0,911770,110,BIZKAIA,507,121,153,619,339,384,1238,...,125,2201,242,36178,99,67161,549,994,575,7118
2,5900,17,ABADIÑO,2,1,0,1,0,1,5,...,1,10,2,155,0,403,2,7,0,35
4,7769,0,ABANTO Y CIERVANA-ABANTO ZIERBENA,1,2,1,3,1,3,3,...,0,8,6,94,1,844,4,7,3,46
6,388,0,AJANGIZ,0,0,0,0,1,0,0,...,0,0,0,2,0,12,0,0,0,0
8,2328,0,ALONSOTEGI,1,0,0,1,57,2,1,...,0,1,1,34,0,166,1,2,1,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,1313,0,ZARATAMO,0,1,0,0,0,0,0,...,0,1,0,25,0,90,0,0,0,4
218,1005,0,ZEANURI,1,0,0,0,1,0,0,...,0,5,0,6,0,31,0,1,0,0
220,860,0,ZEBERIO,0,0,0,0,0,0,1,...,0,2,0,3,0,35,0,0,0,2
222,1226,0,ZIERBENA,0,0,0,0,0,0,1,...,0,8,0,13,0,69,0,1,0,3


In [7]:
datos_perc_df

,ZENTSUA/CENSO,M.C.R.,EREMUAK/AMBITOS,CPE,ADN,IGRE,PCPE-PCPC-P,ALTER,PCTE-ELAK,I.FEM,...,CONTIGO,JUNTS,PACT,PP,SAIN,PODEMOS-IU,PUM+J,R0-LV-GVE,CXE,VOX
1,911770,"0,02",BIZKAIA,"0,09","0,02","0,03","0,1","0,06","0,06","0,21",...,"0,02","0,37","0,04","6,11","0,02","11,35","0,09","0,17","0,1","1,2"
3,5900,"0,42",ABADIÑO,"0,05","0,02",0,"0,02",0,"0,02","0,12",...,"0,02","0,25","0,05","3,86",0,"10,03","0,05","0,17",0,"0,87"
5,7769,0,ABANTO Y CIERVANA-ABANTO ZIERBENA,"0,02","0,04","0,02","0,06","0,02","0,06","0,06",...,0,"0,15","0,11","1,77","0,02","15,87","0,08","0,13","0,06","0,86"
7,388,0,AJANGIZ,0,0,0,0,"0,32",0,0,...,0,0,0,"0,63",0,"3,8",0,0,0,0
9,2328,0,ALONSOTEGI,"0,06",0,0,"0,06","3,33","0,12","0,06",...,0,"0,06","0,06","1,98",0,"9,68","0,06","0,12","0,06","0,58"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,1313,0,ZARATAMO,0,"0,11",0,0,0,0,0,...,0,"0,11",0,"2,77",0,"9,99",0,0,0,"0,44"
219,1005,0,ZEANURI,"0,12",0,0,0,"0,12",0,0,...,0,"0,62",0,"0,75",0,"3,87",0,"0,12",0,0
221,860,0,ZEBERIO,0,0,0,0,0,0,"0,14",...,0,"0,29",0,"0,43",0,"4,99",0,0,0,"0,29"
223,1226,0,ZIERBENA,0,0,0,0,0,0,"0,11",...,0,"0,9",0,"1,46",0,"7,75",0,"0,11",0,"0,34"


In [8]:
datos_perc_df.columns

Index(['ZENTSUA/CENSO', 'M.C.R.', 'EREMUAK/AMBITOS', 'CPE', 'ADN', 'IGRE',
       'PCPE-PCPC-P', 'ALTER', 'PCTE-ELAK', 'I.FEM',
       'ABSTENTZIOA %/% ABSTENCION', 'MIEL', 'ZENBATUTA %/% ESCRUTADO',
       'BALIOGABEAK/NULOS', 'FAC', 'PH', 'ZURIAK/BLANCOS', 'PIRATES/EP',
       'PSE-EE/PSOE', 'AXSI', 'EH-BILDU OR', 'IZQP', 'CEX-CREX-PR', 'EAJ-PNV',
       'VOLT', 'PACMA/ATTAA', 'CV-EC', 'CS', 'CONTIGO', 'JUNTS', 'PACT', 'PP',
       'SAIN', 'PODEMOS-IU', 'PUM+J', 'R0-LV-GVE', 'CXE', 'VOX'],
      dtype='object')

Vamos a obtener los datos asociados a la comunidad. Primero sacamos un dataframe con los datos de la comunidad y dejamos al resto sin este dato.

In [9]:
query = """
CREATE TABLE IF NOT EXISTS PROVINCIA_DIM
(      
    Id INTEGER PRIMARY KEY,
    Nombre TEXT
)
"""

c = con.connection.cursor()
c.execute(query)
con.commit()

In [10]:
query = """
INSERT INTO PROVINCIA_DIM VALUES (1, 'BIZKAIA')
"""

c = con.connection.cursor()
c.execute(query)
con.commit()

In [11]:
# Datos de la propia comunidad
cond = datos_perc_df['EREMUAK/AMBITOS'] == 'BIZKAIA'

datos_perc_df.loc[~cond, 'EREMUAK/AMBITOS']

3                                ABADIÑO
5      ABANTO Y CIERVANA-ABANTO ZIERBENA
7                                AJANGIZ
9                             ALONSOTEGI
11                     AMOREBIETA-ETXANO
                     ...                
217                             ZARATAMO
219                              ZEANURI
221                              ZEBERIO
223                             ZIERBENA
225                      ZIORTZA-BOLIBAR
Name: EREMUAK/AMBITOS, Length: 112, dtype: object

In [12]:
query = """
CREATE TABLE IF NOT EXISTS AMBITO_DIM
(      
    Id INTEGER PRIMARY KEY,
    Nombre TEXT,
    IdProvincia INTEGER,
    FOREIGN KEY (IdProvincia) REFERENCES PROVINCIA_DIM(Id)
)
"""

c = con.connection.cursor()
c.execute(query)
con.commit()

In [19]:
c = con.connection.cursor()
for i, ambito in enumerate(datos_perc_df.loc[~cond, 'EREMUAK/AMBITOS']):

    query = f"""
    INSERT INTO AMBITO_DIM VALUES ({i+2}, '{ambito}', 1)
    """

    c.execute(query)
con.commit()

Vamos ahora con los datos recogidos para cada ámbito. Primeramente existen ciertos datos asociados únicamente al ámbito.

In [20]:
# Datos de la elección en la comunidad
AMBTITO_STATUS_FACT = ['EREMUAK/AMBITOS','ZENTSUA/CENSO', 'ABSTENTZIOA %/% ABSTENCION', 'ZENBATUTA %/% ESCRUTADO', 'BALIOGABEAK/NULOS']

datos_perc_df.loc[~cond, AMBTITO_STATUS_FACT]

,EREMUAK/AMBITOS,ZENTSUA/CENSO,ABSTENTZIOA %/% ABSTENCION,ZENBATUTA %/% ESCRUTADO,BALIOGABEAK/NULOS
3,ABADIÑO,5900,"31,44",100,"0,64"
5,ABANTO Y CIERVANA-ABANTO ZIERBENA,7769,"31,07",100,"0,67"
7,AJANGIZ,388,"18,3",100,"0,32"
9,ALONSOTEGI,2328,"25,86",100,"0,7"
11,AMOREBIETA-ETXANO,14633,"30,68",100,"0,58"
...,...,...,...,...,...
217,ZARATAMO,1313,"30,77",100,"0,88"
219,ZEANURI,1005,"19,4",100,"0,99"
221,ZEBERIO,860,"18,14",100,"0,43"
223,ZIERBENA,1226,"26,67",100,1


In [21]:
comunidades = pd.read_sql("SELECT Id, Nombre FROM AMBITO_DIM", con=con)
comunidades

,Id,Nombre
0,2,ABADIÑO
1,3,ABANTO Y CIERVANA-ABANTO ZIERBENA
2,4,AJANGIZ
3,5,ALONSOTEGI
4,6,AMOREBIETA-ETXANO
...,...,...
107,109,ZARATAMO
108,110,ZEANURI
109,111,ZEBERIO
110,112,ZIERBENA


In [22]:
datos_status = comunidades.set_index("Nombre").join(datos_perc_df[AMBTITO_STATUS_FACT].set_index("EREMUAK/AMBITOS"))
datos_status

,Id,ZENTSUA/CENSO,ABSTENTZIOA %/% ABSTENCION,ZENBATUTA %/% ESCRUTADO,BALIOGABEAK/NULOS
Nombre,,,,,
ABADIÑO,2,5900,"31,44",100,"0,64"
ABANTO Y CIERVANA-ABANTO ZIERBENA,3,7769,"31,07",100,"0,67"
AJANGIZ,4,388,"18,3",100,"0,32"
ALONSOTEGI,5,2328,"25,86",100,"0,7"
AMOREBIETA-ETXANO,6,14633,"30,68",100,"0,58"
...,...,...,...,...,...
ZARATAMO,109,1313,"30,77",100,"0,88"
ZEANURI,110,1005,"19,4",100,"0,99"
ZEBERIO,111,860,"18,14",100,"0,43"


In [23]:
datos_status.dtypes

Id                             int64
ZENTSUA/CENSO                  int64
ABSTENTZIOA %/% ABSTENCION    object
ZENBATUTA %/% ESCRUTADO        int64
BALIOGABEAK/NULOS             object
dtype: object

Debemos interpretar ciertos datos como numéricos cambiándoles antes el separador de miles.

In [24]:
datos_status['ABSTENTZIOA %/% ABSTENCION'] = datos_status['ABSTENTZIOA %/% ABSTENCION'].str.replace(",",".").astype(float)
datos_status['BALIOGABEAK/NULOS'] = datos_status['BALIOGABEAK/NULOS'].str.replace(",",".").astype(float)

In [25]:
datos_status.dtypes

Id                              int64
ZENTSUA/CENSO                   int64
ABSTENTZIOA %/% ABSTENCION    float64
ZENBATUTA %/% ESCRUTADO         int64
BALIOGABEAK/NULOS             float64
dtype: object

In [26]:
query = """
CREATE TABLE IF NOT EXISTS AMBITO_STATUS_FACT
(      
    Id INTEGER,
    Censo INTEGER,
    Abstención DOUBLE,
    Escrutado DOUBLE,
    Nulos DOUBLE,
    Año INTEGER,
    PRIMARY KEY (Id, Año)
)
"""

c = con.connection.cursor()
c.execute(query)
con.commit()

In [27]:
datos_status['Año'] = 2019
datos_status.rename(columns={"ZENTSUA/CENSO" : "Censo", "ABSTENTZIOA %/% ABSTENCION" : "Abstención", "ZENBATUTA %/% ESCRUTADO" : "Escrutado", "BALIOGABEAK/NULOS" : "Nulos"}, inplace=True)
datos_status

,Id,Censo,Abstención,Escrutado,Nulos,Año
Nombre,,,,,,
ABADIÑO,2,5900,31.44,100,0.64,2019
ABANTO Y CIERVANA-ABANTO ZIERBENA,3,7769,31.07,100,0.67,2019
AJANGIZ,4,388,18.30,100,0.32,2019
ALONSOTEGI,5,2328,25.86,100,0.70,2019
AMOREBIETA-ETXANO,6,14633,30.68,100,0.58,2019
...,...,...,...,...,...,...
ZARATAMO,109,1313,30.77,100,0.88,2019
ZEANURI,110,1005,19.40,100,0.99,2019
ZEBERIO,111,860,18.14,100,0.43,2019


In [28]:
datos_status.to_sql('AMBITO_STATUS_FACT', engine, if_exists='append', index=False)

112

In [29]:
pd.read_sql_table("AMBITO_STATUS_FACT", con=con)

,Id,Censo,Abstención,Escrutado,Nulos,Año
0,2,5900,31.44,100.0,0.64,2019
1,3,7769,31.07,100.0,0.67,2019
2,4,388,18.30,100.0,0.32,2019
3,5,2328,25.86,100.0,0.70,2019
4,6,14633,30.68,100.0,0.58,2019
...,...,...,...,...,...,...
107,109,1313,30.77,100.0,0.88,2019
108,110,1005,19.40,100.0,0.99,2019
109,111,860,18.14,100.0,0.43,2019
110,112,1226,26.67,100.0,1.00,2019


Validemos si el dato agregado coincide.

In [24]:
pd.read_sql("SELECT SUM(Censo) FROM AMBITO_STATUS_FACT", con=con)

,SUM(Censo)
0,911770.0


In [25]:
datos_perc_df[cond]

,ZENTSUA/CENSO,M.C.R.,EREMUAK/AMBITOS,CPE,ADN,IGRE,PCPE-PCPC-P,ALTER,PCTE-ELAK,I.FEM,...,CONTIGO,JUNTS,PACT,PP,SAIN,PODEMOS-IU,PUM+J,R0-LV-GVE,CXE,VOX
1,911770,"0,02",BIZKAIA,"0,09","0,02","0,03","0,1","0,06","0,06","0,21",...,"0,02","0,37","0,04","6,11","0,02","11,35","0,09","0,17","0,1","1,2"


Podemos corregir las tablas añadiendo claves foráneas si se nos hubieran olvidado en la definición.

In [30]:
query = """
ALTER TABLE AMBITO_STATUS_FACT ADD CONSTRAINT fk_ambito_id FOREIGN KEY (Id) REFERENCES AMBITO_DIM(Id);
"""

c = con.connection.cursor()
c.execute(query)
con.commit()

### Partidos

In [31]:
PARTIDOS = [col for col in datos_perc_df.columns if col not in AMBTITO_STATUS_FACT]

In [33]:
tabla_partidos = pd.DataFrame({"Id" : range(1, len(PARTIDOS)+1), "Partido": PARTIDOS})
tabla_partidos.dtypes

Id          int64
Partido    object
dtype: object

In [35]:
query = """
CREATE TABLE IF NOT EXISTS PARTIDOS_DIM
(      
    Id INTEGER PRIMARY KEY,
    Partido TEXT
)
"""

c = con.connection.cursor()
c.execute(query)
con.commit()

In [36]:
tabla_partidos.to_sql("PARTIDOS_DIM", engine, index=False,  if_exists='append')

33

In [37]:
pd.read_sql_table("PARTIDOS_DIM", engine)

,Id,Partido
0,1,M.C.R.
1,2,CPE
2,3,ADN
3,4,IGRE
4,5,PCPE-PCPC-P
5,6,ALTER
6,7,PCTE-ELAK
7,8,I.FEM
8,9,MIEL
9,10,FAC


In [38]:
datos_a_pivotar = datos_cantidad_df[['EREMUAK/AMBITOS']+PARTIDOS]
datos_a_pivotar = datos_a_pivotar[datos_a_pivotar["EREMUAK/AMBITOS"] != "BIZKAIA"].copy()
datos_a_pivotar

,EREMUAK/AMBITOS,M.C.R.,CPE,ADN,IGRE,PCPE-PCPC-P,ALTER,PCTE-ELAK,I.FEM,MIEL,...,CONTIGO,JUNTS,PACT,PP,SAIN,PODEMOS-IU,PUM+J,R0-LV-GVE,CXE,VOX
2,ABADIÑO,17,2,1,0,1,0,1,5,2,...,1,10,2,155,0,403,2,7,0,35
4,ABANTO Y CIERVANA-ABANTO ZIERBENA,0,1,2,1,3,1,3,3,1,...,0,8,6,94,1,844,4,7,3,46
6,AJANGIZ,0,0,0,0,0,1,0,0,0,...,0,0,0,2,0,12,0,0,0,0
8,ALONSOTEGI,0,1,0,0,1,57,2,1,0,...,0,1,1,34,0,166,1,2,1,10
10,AMOREBIETA-ETXANO,4,10,0,4,17,10,3,149,5,...,0,25,1,202,0,1001,5,28,5,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,ZARATAMO,0,0,1,0,0,0,0,0,0,...,0,1,0,25,0,90,0,0,0,4
218,ZEANURI,0,1,0,0,0,1,0,0,0,...,0,5,0,6,0,31,0,1,0,0
220,ZEBERIO,0,0,0,0,0,0,0,1,0,...,0,2,0,3,0,35,0,0,0,2
222,ZIERBENA,0,0,0,0,0,0,0,1,0,...,0,8,0,13,0,69,0,1,0,3


Sustituimos el nombre por el identificador en la base de datos.

In [39]:
datos_votos = comunidades.set_index("Nombre").join(datos_a_pivotar.set_index("EREMUAK/AMBITOS")).reset_index(drop=True).set_index("Id")
datos_votos

,M.C.R.,CPE,ADN,IGRE,PCPE-PCPC-P,ALTER,PCTE-ELAK,I.FEM,MIEL,FAC,...,CONTIGO,JUNTS,PACT,PP,SAIN,PODEMOS-IU,PUM+J,R0-LV-GVE,CXE,VOX
Id,,,,,,,,,,,,,,,,,,,,,
2,17,2,1,0,1,0,1,5,2,0,...,1,10,2,155,0,403,2,7,0,35
3,0,1,2,1,3,1,3,3,1,0,...,0,8,6,94,1,844,4,7,3,46
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,2,0,12,0,0,0,0
5,0,1,0,0,1,57,2,1,0,0,...,0,1,1,34,0,166,1,2,1,10
6,4,10,0,4,17,10,3,149,5,1,...,0,25,1,202,0,1001,5,28,5,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,0,0,1,0,0,0,0,0,0,0,...,0,1,0,25,0,90,0,0,0,4
110,0,1,0,0,0,1,0,0,0,0,...,0,5,0,6,0,31,0,1,0,0
111,0,0,0,0,0,0,0,1,0,0,...,0,2,0,3,0,35,0,0,0,2


Hay que hacer un cambio importante aquí:

* Renombramos las columnas tomando su número de identificación para cada partido.
* Los apilamos por ámbito y partido.
* Cambiamos el nombre para que coincida con los nombres de base de datos
* Añadimos la columna de la convocatoria
* Eliminamos información vacía

In [40]:
datos_votos.rename(columns=tabla_partidos.set_index("Partido").to_dict()["Id"], inplace=True)
datos_votos_df = pd.DataFrame(data=datos_votos.stack().reset_index())
datos_votos_df.columns=["IdAmbito","IdPartido","Votos"]
datos_votos_df["Año"] = 2019
datos_votos_df = datos_votos_df.loc[datos_votos_df["Votos"] != '0', :]
datos_votos_df

,IdAmbito,IdPartido,Votos,Año
0,2,1,17,2019
1,2,2,2,2019
2,2,3,1,2019
4,2,5,1,2019
6,2,7,1,2019
...,...,...,...,...
3678,113,16,153,2019
3681,113,19,95,2019
3685,113,23,1,2019
3689,113,27,2,2019


In [41]:
query = """
CREATE TABLE IF NOT EXISTS VOTOS_FACT
(      
    IdAmbito INTEGER,
    IdPartido INTEGER,
    Año INTEGER,
    Votos INTEGER,
    PRIMARY KEY (IdAmbito, IdPartido, Año)
)
"""

c = con.connection.cursor()
c.execute(query)
con.commit()

In [42]:
datos_votos_df.to_sql("VOTOS_FACT", engine, index=False,  if_exists='append')

1944

Vemos que el número de filas insertadas coincide con las del dataframe. Vamos a introducir restricciones para evitar problemas a la hora de borrar o insertar datos.

In [43]:
query = """
ALTER TABLE VOTOS_FACT ADD CONSTRAINT fk_ambito_votos_id FOREIGN KEY (IdAmbito) REFERENCES AMBITO_DIM(Id);
"""

c = con.connection.cursor()
c.execute(query)
con.commit()

In [44]:
query = """
ALTER TABLE VOTOS_FACT ADD CONSTRAINT fk_partido_votos_id FOREIGN KEY (IdPartido) REFERENCES PARTIDOS_DIM(Id);
"""

c = con.connection.cursor()
c.execute(query)
con.commit()

In [45]:
import pandas as pd

pd.read_sql_query("SELECT * FROM information_schema.tables WHERE TABLE_SCHEMA = 'elecciones'", con=con)

,TABLE_CATALOG,TABLE_SCHEMA,TABLE_NAME,TABLE_TYPE,ENGINE,VERSION,ROW_FORMAT,TABLE_ROWS,AVG_ROW_LENGTH,DATA_LENGTH,...,INDEX_LENGTH,DATA_FREE,AUTO_INCREMENT,CREATE_TIME,UPDATE_TIME,CHECK_TIME,TABLE_COLLATION,CHECKSUM,CREATE_OPTIONS,TABLE_COMMENT
0,def,elecciones,AMBITO_DIM,BASE TABLE,InnoDB,10,Dynamic,112,146,16384,...,16384,0,None,2024-06-29 09:53:55,NaT,None,utf8mb4_0900_ai_ci,None,,
1,def,elecciones,AMBITO_STATUS_FACT,BASE TABLE,InnoDB,10,Dynamic,112,146,16384,...,0,0,None,2024-06-29 10:05:56,NaT,None,utf8mb4_0900_ai_ci,None,,
2,def,elecciones,PARTIDOS_DIM,BASE TABLE,InnoDB,10,Dynamic,33,496,16384,...,0,0,None,2024-06-29 10:16:05,2024-06-29 10:16:06,None,utf8mb4_0900_ai_ci,None,,
3,def,elecciones,PROVINCIA_DIM,BASE TABLE,InnoDB,10,Dynamic,0,0,16384,...,0,0,None,2024-06-29 09:53:54,2024-06-29 09:53:55,None,utf8mb4_0900_ai_ci,None,,
4,def,elecciones,VOTOS_FACT,BASE TABLE,InnoDB,10,Dynamic,1944,58,114688,...,81920,0,None,2024-06-29 10:23:18,NaT,None,utf8mb4_0900_ai_ci,None,,


# Extensión del modelo

Podemos ahora extender el modelo sin tocar los datos para realizar agregaciones de distinto orden. Por ejemplo, asociar los partidos a una orientación política.

In [46]:
query = """
CREATE TABLE IF NOT EXISTS ORIENTACION_DIM
(      
    Id INTEGER PRIMARY KEY,
    Orientacion TEXT
)
"""

c = con.connection.cursor()
c.execute(query)
con.commit()

In [47]:
query = """
INSERT INTO ORIENTACION_DIM VALUES (1, 'Izquierda'), (2, 'Derecha')
"""

c = con.connection.cursor()
c.execute(query)
con.commit()

In [48]:
pd.read_sql_table("ORIENTACION_DIM", con=con)

,Id,Orientacion
0,1,Izquierda
1,2,Derecha


Ahora relacionamos los partidos con su orientación. Primero deberemos añadir una columna que indique de que pie cojean.

In [49]:
query = """
ALTER TABLE PARTIDOS_DIM ADD COLUMN IdOrientacion INTEGER;
"""

c = con.connection.cursor()
c.execute(query)
con.commit()

Luego una restricción que relaciona los valores de una columna con la otra.

In [50]:
query = """
ALTER TABLE PARTIDOS_DIM ADD CONSTRAINT fk_partido_orientacion_id FOREIGN KEY (IdOrientacion) REFERENCES ORIENTACION_DIM(Id);
"""

c = con.connection.cursor()
c.execute(query)
con.commit()

Asumiremos que todos los partidos previamente indicados son de Izquierdas.

In [51]:
query = """
UPDATE PARTIDOS_DIM SET IdOrientacion = 1
"""

c = con.connection.cursor()
c.execute(query)
con.commit()

Menos dos: PP y VOX

In [52]:
query = """
UPDATE PARTIDOS_DIM SET IdOrientacion = 2 WHERE Partido in ('VOX', 'PP')
"""

c = con.connection.cursor()
c.execute(query)
con.commit()

Veamos cual es el resultado a distintos niveles de agregación.

In [53]:
pd.read_sql_query("""
SELECT SUM(Votos), ad.Nombre as Nombre
FROM AMBITO_DIM ad
INNER JOIN VOTOS_FACT vf ON vf.IdAmbito = ad.Id
GROUP BY Nombre 
""", con=con)

,SUM(Votos),Nombre
0,4019.0,ABADIÑO
1,5319.0,ABANTO Y CIERVANA-ABANTO ZIERBENA
2,316.0,AJANGIZ
3,1714.0,ALONSOTEGI
4,10085.0,AMOREBIETA-ETXANO
...,...,...
107,901.0,ZARATAMO
108,802.0,ZEANURI
109,701.0,ZEBERIO
110,890.0,ZIERBENA


In [54]:
pd.read_sql_query(""" 
SELECT SUM(Votos), pd.Nombre as Nombre 
FROM AMBITO_DIM ad
INNER JOIN VOTOS_FACT vf ON vf.IdAmbito = ad.Id
INNER JOIN PROVINCIA_DIM pd ON pd.Id = ad.IdProvincia 
GROUP BY Nombre 
""", con=con)

,SUM(Votos),Nombre
0,591913.0,BIZKAIA


In [55]:
pd.read_sql_query("""
SELECT SUM(vf.Votos), pd.Partido as Partido
FROM PARTIDOS_DIM pd
INNER JOIN VOTOS_FACT vf ON vf.IdPartido = pd.Id
GROUP BY Partido
""", con=con)

,SUM(vf.Votos),Partido
0,110.0,M.C.R.
1,507.0,CPE
2,121.0,ADN
3,153.0,IGRE
4,619.0,PCPE-PCPC-P
5,339.0,ALTER
6,384.0,PCTE-ELAK
7,1238.0,I.FEM
8,79.0,MIEL
9,156.0,FAC


In [56]:
pd.read_sql_query("""
SELECT SUM(vf.Votos), od.Orientacion as Orientacion
FROM PARTIDOS_DIM pd
INNER JOIN VOTOS_FACT vf ON vf.IdPartido = pd.Id
INNER JOIN ORIENTACION_DIM od ON od.Id = pd.IdOrientacion
GROUP BY Orientacion
""", con=con)

,SUM(vf.Votos),Orientacion
0,548617.0,Izquierda
1,43296.0,Derecha


# Extensión de datos

España: https://resultados.eleccioneseuropeas2024.es/es/descargas

In [51]:
#!pip install tabula-py

In [57]:
import tabula

# extract all the tables in the PDF file
df = tabula.read_pdf("datos2024/dossier_es_pro.pdf", pages="all")

Failed to import jpype dependencies. Fallback to subprocess.
No module named 'jpype'


In [58]:
len(df)

1818

In [59]:
indices_con_datos = [i for i, df_val in enumerate(df) if len(df_val) > 0]

Tomamos un índice al azar.

In [60]:
df[1792] # Zizurkil, Zumaia y Zumarraga

,Unnamed: 0,Zizurkil,Unnamed: 1,Zumaia,Unnamed: 2,Zumarraga
0,Escrutado,NaN,100%,NaN,100%,100%
1,Participación,1.040,"47,18%",4.238,"53,67%","3.730 49,62%"
2,Abstención,1.164,"52,81%",3.657,"46,32%","3.787 50,37%"
3,Votos nulos,2.000,"0,19%",9.000,"0,21%","16 0,42%"
4,Votos en blanco,10.000,"0,96%",22.000,"0,52%","22 0,59%"
5,EH BILDU-ORAIN,NaN,NaN,NaN,NaN,NaN
6,NaN,477.000,"45,95%",1.858,"43,93%","1.009 27,16%"
7,ERREPUBLIKAK,NaN,NaN,NaN,NaN,NaN
8,PSOE,208.000,"20,03%",816.000,"19,29%","1.142 30,74%"
9,EAJ-PNV,192.000,"18,49%",1.090,"25,77%","673 18,12%"


Habría que limpiar más a fondo la información de Zumarraga pero por el momento prescindiremos de ella.

In [61]:
df_zzz = df[1792].copy()

df_zzz = df_zzz.loc[5:, :]
df_zzz.columns = ["Partido", "ZIZURKIL", "Eliminar", "ZUMAIA", "Eliminar", "Eliminar"]
df_zzz.drop(columns="Eliminar", inplace=True)
df_zzz.dropna(inplace=True)

In [62]:
new_data = df_zzz.set_index('Partido').stack()
new_data

Partido                 
PSOE            ZIZURKIL    208.00
                ZUMAIA      816.00
EAJ-PNV         ZIZURKIL    192.00
                ZUMAIA        1.09
PP              ZIZURKIL     52.00
                             ...  
EXTREMEÑOS      ZUMAIA        0.00
FE de las JONS  ZIZURKIL      0.00
                ZUMAIA        0.00
PREPAL          ZIZURKIL      1.00
                ZUMAIA        0.00
Length: 64, dtype: float64

In [63]:
new_data = new_data[new_data > 0]
new_data

Partido                     
PSOE                ZIZURKIL    208.00
                    ZUMAIA      816.00
EAJ-PNV             ZIZURKIL    192.00
                    ZUMAIA        1.09
PP                  ZIZURKIL     52.00
                    ZUMAIA      135.00
SUMAR               ZIZURKIL     28.00
                    ZUMAIA       89.00
PODEMOS             ZIZURKIL     31.00
                    ZUMAIA       80.00
VOX                 ZIZURKIL     12.00
                    ZUMAIA       44.00
PACMA               ZIZURKIL      2.00
                    ZUMAIA       13.00
FO                  ZIZURKIL      5.00
                    ZUMAIA       13.00
IE                  ZIZURKIL      3.00
                    ZUMAIA       12.00
ESCAÑOS EN BLANCO   ZIZURKIL      1.00
                    ZUMAIA        5.00
PFAC                ZUMAIA        2.00
Cs                  ZIZURKIL      1.00
                    ZUMAIA        3.00
JUNTS UE            ZUMAIA        3.00
PCPE/PCPC           ZUMAIA        1

Insertaremos la provincia de estos dos nuevos ámbitos.

In [64]:
query = """
INSERT INTO PROVINCIA_DIM VALUES (2, 'GIPUZKOA')
"""

c = con.connection.cursor()
c.execute(query)
con.commit()

In [65]:
import pandas as pd

pd.read_sql_query("""
SELECT MAX(Id) FROM AMBITO_DIM
""", con=con)

,MAX(Id)
0,113


Tendremos que emplear un número superior a este.

In [66]:
query = """
INSERT INTO AMBITO_DIM VALUES (114, 'ZIZURKIL', 2), (115, 'ZUMAIA', 2)
"""

c = con.connection.cursor()
c.execute(query)
con.commit()

In [67]:
partidos = pd.read_sql_query(""" 
SELECT Id, Partido FROM PARTIDOS_DIM
""", con=con)
partidos

,Id,Partido
0,1,M.C.R.
1,2,CPE
2,3,ADN
3,4,IGRE
4,5,PCPE-PCPC-P
5,6,ALTER
6,7,PCTE-ELAK
7,8,I.FEM
8,9,MIEL
9,10,FAC


In [68]:
part_dict = partidos.set_index("Partido").to_dict()["Id"]
part_dict["PP"]

27

In [69]:
max(partidos["Id"])

33

In [70]:
max_id_partidos = max(partidos["Id"])+1
ambitos = {"ZIZURKIL": 114, "ZUMAIA": 115}

c = con.connection.cursor()
for key, votos in new_data.items():
    partido_str = key[0]

    if partido_str not in part_dict:
        query = f"""
        INSERT INTO PARTIDOS_DIM VALUES ({max_id_partidos}, '{partido_str}', 1)
        """
        c.execute(query)

        part_dict[partido_str] = max_id_partidos
        max_id_partidos += 1

con.commit()

In [71]:
partidos = pd.read_sql_query(""" 
SELECT Id, Partido FROM PARTIDOS_DIM
""", con=con)
partidos

,Id,Partido
0,1,M.C.R.
1,2,CPE
2,3,ADN
3,4,IGRE
4,5,PCPE-PCPC-P
5,6,ALTER
6,7,PCTE-ELAK
7,8,I.FEM
8,9,MIEL
9,10,FAC


In [72]:
c = con.connection.cursor()
for key, votos in new_data.items():
    partido_str = key[0]

    id_partido = part_dict[partido_str]
    ambito = ambitos[key[1]]

    query = f"""
    INSERT INTO VOTOS_FACT VALUES ({ambito}, {id_partido}, 2024, {votos})
    """

    c.execute(query)

con.commit()

In [73]:
pd.read_sql_query(""" 
SELECT pd.Id, Partido, Orientacion 
FROM PARTIDOS_DIM pd
INNER JOIN ORIENTACION_DIM od ON od.Id = pd.IdOrientacion
""", con=con)

,Id,Partido,Orientacion
0,1,M.C.R.,Izquierda
1,2,CPE,Izquierda
2,3,ADN,Izquierda
3,4,IGRE,Izquierda
4,5,PCPE-PCPC-P,Izquierda
5,6,ALTER,Izquierda
6,7,PCTE-ELAK,Izquierda
7,8,I.FEM,Izquierda
8,9,MIEL,Izquierda
9,10,FAC,Izquierda


# Correcciones

Podemos probar si cambiando el nombre podemos visualizar mejor el mapa.

In [74]:
query = """
UPDATE AMBITO_DIM SET Nombre = 'ZUMAYA' WHERE Id = 115;
"""

c = con.connection.cursor()
c.execute(query)
con.commit()

In [75]:
con.close()

Nos hemos quedado lejos pero con algo más de trabajo seguro que conseguiríamos montar [algo así](https://results.elections.europa.eu/es/).